# Recommender Systems 2018/19

### Practice session on BPR-MF

### Course survey on the challenge! https://goo.gl/forms/dgWszJeJW8D9I1Q62


## Recap on BPR
S.Rendle et al. BPR: Bayesian Personalized Ranking from Implicit Feedback. UAI2009

The usual approach for item recommenders is to predict a personalized score $\hat{x}_{ui}$ for an item that reflects the preference of the user for the item. Then the items are ranked by sorting them according to that score.

Machine learning approaches are tipically fit by using observed items as a positive sample and missing ones for the negative class. A perfect model would thus be useless, as it would classify as negative (non-interesting) all the items that were non-observed at training time. The only reason why such methods work is regularization.

BPR use a different approach. The training dataset is composed by triplets $(u,i,j)$ representing that user u is assumed to prefer i over j. For an implicit dataset this means that u observed i but not j:
$$D_S := \{(u,i,j) \mid i \in I_u^+ \wedge j \in I \setminus I_u^+\}$$

### BPR-OPT
A machine learning model can be represented by a parameter vector $\Theta$ which is found at fitting time. BPR wants to find the parameter vector that is most probable given the desired, but latent, preference structure $>_u$:
$$p(\Theta \mid >_u) \propto p(>_u \mid \Theta)p(\Theta) $$
$$\prod_{u\in U} p(>_u \mid \Theta) = \dots = \prod_{(u,i,j) \in D_S} p(i >_u j \mid \Theta) $$

The probability that a user really prefers item $i$ to item $j$ is defined as:
$$ p(i >_u j \mid \Theta) := \sigma(\hat{x}_{uij}(\Theta)) $$
Where $\sigma$ represent the logistic sigmoid and $\hat{x}_{uij}(\Theta)$ is an arbitrary real-valued function of $\Theta$ (the output of your arbitrary model).


To complete the Bayesian setting, we define a prior density for the parameters:
$$p(\Theta) \sim N(0, \Sigma_\Theta)$$
And we can now formulate the maximum posterior estimator:
$$BPR-OPT := \log p(\Theta \mid >_u) $$
$$ = \log p(>_u \mid \Theta) p(\Theta) $$
$$ = \log \prod_{(u,i,j) \in D_S} \sigma(\hat{x}_{uij})p(\Theta) $$
$$ = \sum_{(u,i,j) \in D_S} \log \sigma(\hat{x}_{uij}) + \log p(\Theta) $$
$$ = \sum_{(u,i,j) \in D_S} \log \sigma(\hat{x}_{uij}) - \lambda_\Theta ||\Theta||^2 $$

Where $\lambda_\Theta$ are model specific regularization parameters.

### BPR learning algorithm
Once obtained the log-likelihood, we need to maximize it in order to find our obtimal $\Theta$. As the crierion is differentiable, gradient descent algorithms are an obvious choiche for maximization.

Gradient descent comes in many fashions, you can find an overview on my master thesis https://www.politesi.polimi.it/bitstream/10589/133864/3/tesi.pdf on pages 18-19-20 (I'm linking my thesis just because I'm sure of what it's written there, many posts you can find online contain some error). A nice post about momentum is available here https://distill.pub/2017/momentum/

The basic version of gradient descent consists in evaluating the gradient using all the available samples and then perform a single update. The problem with this is, in our case, that our training dataset is very skewed. Suppose an item i is very popular. Then we habe many terms of the form $\hat{x}_{uij}$ in the loss because for many users u the item i is compared against all negative items j.

The other popular approach is stochastic gradient descent, where for each training sample an update is performed. This is a better approach, but the order in which the samples are traversed is crucial. To solve this issue BPR uses a stochastic gradient descent algorithm that choses the triples randomly.

The gradient of BPR-OPT with respect to the model parameters is: 
$$\frac{\partial BPR-OPT}{\partial \Theta} = \sum_{(u,i,j) \in D_S} \frac{\partial}{\partial \Theta} \log \sigma (\hat{x}_{uij}) - \lambda_\Theta \frac{\partial}{\partial\Theta} || \Theta ||^2$$
$$ =  \sum_{(u,i,j) \in D_S} \frac{-e^{-\hat{x}_{uij}}}{1+e^{-\hat{x}_{uij}}} \frac{\partial}{\partial \Theta}\hat{x}_{uij} - \lambda_\Theta \Theta $$

### BPR-MF

In order to practically apply this learning schema to an existing algorithm, we first split the real valued preference term: $\hat{x}_{uij} := \hat{x}_{ui} − \hat{x}_{uj}$. And now we can apply any standard collaborative filtering model that predicts $\hat{x}_{ui}$.

The problem of predicting $\hat{x}_{ui}$ can be seen as the task of estimating a matrix $X:U×I$. With matrix factorization teh target matrix $X$ is approximated by the matrix product of two low-rank matrices $W:|U|\times k$ and $H:|I|\times k$:
$$X := WH^t$$
The prediction formula can also be written as:
$$\hat{x}_{ui} = \langle w_u,h_i \rangle = \sum_{f=1}^k w_{uf} \cdot h_{if}$$
Besides the dot product ⟨⋅,⋅⟩, in general any kernel can be used.

We can now specify the derivatives:
$$ \frac{\partial}{\partial \theta} \hat{x}_{uij} = \begin{cases}
(h_{if} - h_{jf}) \text{ if } \theta=w_{uf}, \\
w_{uf} \text{ if } \theta = h_{if}, \\
-w_{uf} \text{ if } \theta = h_{jf}, \\
0 \text{ else }
\end{cases} $$

Which basically means: user $u$ prefer $i$ over $j$, let's do the following:
- Increase the relevance (according to $u$) of features belonging to $i$ but not to $j$ and vice-versa
- Increase the relevance of features assigned to $i$
- Decrease the relevance of features assigned to $j$

We're now ready to look at some code!

In [1]:
from urllib.request import urlretrieve
import zipfile

# skip the download
#urlretrieve ("http://files.grouplens.org/datasets/movielens/ml-10m.zip", "movielens_10m.zip")
dataFile = zipfile.ZipFile("movielens_10m.zip")
URM_path = dataFile.extract("ml-10M100K/ratings.dat")
URM_file = open(URM_path, 'r')


def rowSplit (rowString):
    
    split = rowString.split("::")
    split[3] = split[3].replace("\n","")
    
    split[0] = int(split[0])
    split[1] = int(split[1])
    split[2] = float(split[2])
    split[3] = int(split[3])
    
    result = tuple(split)
    
    return result


URM_file.seek(0)
URM_tuples = []

for line in URM_file:
   URM_tuples.append(rowSplit (line))

userList, itemList, ratingList, timestampList = zip(*URM_tuples)

userList = list(userList)
itemList = list(itemList)
ratingList = list(ratingList)
timestampList = list(timestampList)

import scipy.sparse as sps

URM_all = sps.coo_matrix((ratingList, (userList, itemList)))
URM_all = URM_all.tocsr()



from data_splitter import train_test_holdout


URM_train, URM_test = train_test_holdout(URM_all, train_perc = 0.8)

### MF Computing prediction

### In a MF model you have two matrices, one with a row per user and the other with a column per item. The other dimension, columns for the first one and rows for the second one is called latent factors

In [4]:
num_factors = 100

n_users, n_items = URM_train.shape

In [5]:
import numpy as np

user_factors = np.random.random((n_users, num_factors))

item_factors = np.random.random((n_items, num_factors))

### To compute the prediction we have to muliply the user factors to the item factors

In [16]:
item_index = 15
user_index = 42

prediction = np.dot(user_factors[user_index,:], item_factors[item_index,:])

print("Prediction is {:.2f}".format(prediction))

Prediction is 23.73


# Train a MF MSE model

### Use SGD as we saw for SLIM

In [5]:
test_data = 5
learning_rate = 1e-2
regularization = 1e-3

gradient = test_data - prediction

print("Prediction error is {:.2f}".format(gradient))

Prediction error is 3.05


In [6]:
# Copy original value to avoid messing up the updates
H_i = item_factors[item_index,:]
W_u = user_factors[user_index,:]

user_factors[user_index,:] += learning_rate * (gradient * H_i - regularization * W_u)
item_factors[item_index,:] += learning_rate * (gradient * W_u - regularization * H_i)


In [7]:
prediction = np.dot(user_factors[user_index,:], item_factors[item_index,:])

print("Prediction after the update is {:.2f}".format(prediction))
print("Prediction error is {:.2f}".format(test_data - prediction))

Prediction after the update is 2.12
Prediction error is 2.88


### WARNING: Initialization must be done with random non-zero values ... otherwise

In [8]:
user_factors = np.zeros((n_users, num_factors))

item_factors = np.zeros((n_items, num_factors))

In [9]:
prediction = np.dot(user_factors[user_index,:], item_factors[item_index,:])

print("Prediction is {:.2f}".format(prediction))

gradient = test_data - prediction

print("Prediction error is {:.2f}".format(gradient))

Prediction is 0.00
Prediction error is 5.00


In [10]:
H_i = item_factors[item_index,:]
W_u = user_factors[user_index,:]

user_factors[user_index,:] += learning_rate * (gradient * H_i - regularization * W_u)
item_factors[item_index,:] += learning_rate * (gradient * W_u - regularization * H_i)


In [11]:
prediction = np.dot(user_factors[user_index,:], item_factors[item_index,:])

print("Prediction after the update is {:.2f}".format(prediction))
print("Prediction error is {:.2f}".format(test_data - prediction))

Prediction after the update is 0.00
Prediction error is 5.00


### Since the updates multiply the gradient and the latent factors, if those are zero the SGD will never be able to move from that point

# Train a MF BPR model

## The basics are the same, except for how we compute the gradient, we have to sample a triplet

In [12]:
URM_mask = URM_train.copy()
URM_mask.data[URM_mask.data <= 3] = 0

URM_mask.eliminate_zeros()

# Extract users having at least one interaction to choose from
eligibleUsers = []

for user_id in range(n_users):

    start_pos = URM_mask.indptr[user_id]
    end_pos = URM_mask.indptr[user_id+1]

    if len(URM_mask.indices[start_pos:end_pos]) > 0:
        eligibleUsers.append(user_id)
                
                

def sampleTriplet():
    
    # By randomly selecting a user in this way we could end up 
    # with a user with no interactions
    #user_id = np.random.randint(0, n_users)
    
    user_id = np.random.choice(eligibleUsers)
    
    # Get user seen items and choose one
    userSeenItems = URM_mask[user_id,:].indices
    pos_item_id = np.random.choice(userSeenItems)

    negItemSelected = False

    # It's faster to just try again then to build a mapping of the non-seen items
    while (not negItemSelected):
        neg_item_id = np.random.randint(0, n_items)

        if (neg_item_id not in userSeenItems):
            
            negItemSelected = True

    return user_id, pos_item_id, neg_item_id


In [13]:
#USER, POSITIVE, NEGATIVE.  ITEMS

for _ in range(10):
    print(sampleTriplet())

(62193, 798, 19585)
(70514, 7, 26389)
(45194, 3255, 61872)
(67604, 1466, 2096)
(69865, 3275, 24910)
(36657, 281, 13051)
(43716, 780, 53689)
(55536, 4672, 1139)
(68467, 6377, 8112)
(47261, 1261, 31021)


In [14]:
user_factors = np.random.random((n_users, num_factors))
item_factors = np.random.random((n_items, num_factors))

In [15]:
user_id, positive_item, negative_item = sampleTriplet()

print(user_id, positive_item, negative_item)

52015 110 46652


In [16]:
x_uij = np.dot(user_factors[user_id, :], (item_factors[positive_item,:] - item_factors[negative_item,:]))

x_uij

0.369793948804664

In [17]:
sigmoid_item = 1 / (1 + np.exp(x_uij))

sigmoid_item

0.40859081173853357

### When using BPR we have to update three components, the user factors and the item factors of both the positive and negative item

In [18]:

H_i = item_factors[positive_item,:]
H_j = item_factors[negative_item,:]
W_u = user_factors[user_id,:]


user_factors[user_index,:] += learning_rate * (sigmoid_item * ( H_i - H_j ) - regularization * W_u)
item_factors[positive_item,:] += learning_rate * (sigmoid_item * ( W_u ) - regularization * H_i)
item_factors[negative_item,:] += learning_rate * (sigmoid_item * (-W_u ) - regularization * H_j)


In [ ]:
x_uij = np.dot(user_factors[user_id, :], (item_factors[positive_item,:] - item_factors[negative_item,:]))

x_uij

0.4029981504158613

In [ ]:
## How to rank items with MF ?

## Compute the prediction for all items and rank them

item_scores = np.dot(user_factors[user_index,:], item_factors.T)
item_scores

array([3.66240547, 2.68317025, 2.30127807, ..., 2.90799558, 2.91489822,
       2.28287199])

In [ ]:
item_scores.shape

(65134,)

## Early stopping, how to used and when it is needed

### Problem, how many epochs? 5, 10, 150, 2487 ?

### We could try different values in increasing order: 5, 10, 15, 20, 25...
### However, in this way we would train up to a point, test and then discard the model, to re-train it again up to that same point and then some more... not a good idea.

### Early stopping! 
* Train the model up to a certain number of epochs, say 5
* Compute the recommendation quality on the validation set
* Train for other 5 epochs
* Compute the recommendation quality on the validation set AND compare it with the previous one. If better, then we have another best model, if not, go ahead...
* Repeat until you have either reached the max number of epoch you want to allow (e.g., 300) or a certain number of contiguous validation seps have not updated te best model

### Advantages:
* Easy to implement, we already have all that is required, a train function, a predictor function and an evaluator
* MUCH faster than retraining everything from the beginning
* Often allows to reach even better solutions

### Challenges:
* The evaluation step may be very slow compared to the time it takes to re-train the model

# Train a PureSVD model

### As opposed to the previous ones, PureSVD relies on the SVD decomposition of the URM, which is an easily available function

In [1]:
import traceback, os
import scipy.sparse
URM_all = scipy.sparse.load_npz('URM_all_matrix.npz')
ICM_all = scipy.sparse.load_npz('ICM_all_matrix.npz')
URM_train = scipy.sparse.load_npz('URM_train_matrix.npz')
URM_test = scipy.sparse.load_npz('URM_test_matrix.npz')

In [2]:
from sklearn.utils.extmath import randomized_svd

# Other SVDs are also available, like from sklearn.decomposition import TruncatedSVD

In [7]:
U, Sigma, VT = randomized_svd(URM_train,
              n_components=num_factors,
              #n_iter=5,
              random_state=None)

In [8]:
U

array([[ 1.36627032e-04, -1.02436585e-04, -2.59348639e-05, ...,
        -7.07183581e-04, -1.60134672e-04, -2.52842909e-05],
       [ 2.54179563e-03, -3.41401232e-03, -2.02794893e-03, ...,
        -4.15893249e-04,  3.21816392e-03, -4.71935063e-03],
       [ 1.55282076e-04, -5.75937734e-04, -1.52631890e-04, ...,
         1.57135764e-04, -8.44233271e-04,  4.68803424e-04],
       ...,
       [ 4.18034156e-03, -3.87202721e-03, -2.29676599e-03, ...,
        -6.51662157e-04, -3.41256434e-04,  1.70713196e-03],
       [ 3.25880451e-03, -3.20144001e-04,  3.40756186e-04, ...,
        -2.97416396e-03, -1.50444644e-03,  6.20581656e-03],
       [ 8.45179879e-03, -1.04611113e-02, -4.17630174e-03, ...,
         5.12132991e-03,  5.98923150e-03,  2.37429184e-03]])

In [9]:
U.shape

(50446, 100)

In [10]:
Sigma

array([136.34340895, 107.08019788,  98.28755243,  95.00801674,
        90.8362704 ,  85.43678189,  76.76293765,  69.26126842,
        68.58360878,  68.29430423,  67.06304444,  65.85364616,
        64.1343918 ,  60.98944089,  59.83165091,  58.42113869,
        57.30650465,  56.6233953 ,  54.58954357,  54.3839567 ,
        53.39075328,  52.9868973 ,  52.34160765,  51.5630173 ,
        50.87513463,  50.25528566,  50.0119143 ,  48.93039761,
        48.09262602,  47.92621976,  46.86989776,  46.75942708,
        46.19205583,  45.60913482,  45.14380376,  44.87749341,
        44.57652779,  44.23674792,  44.02459877,  43.71687246,
        43.23113151,  43.0616171 ,  42.94469899,  42.75057804,
        42.60650943,  42.51779875,  42.12742848,  41.92577775,
        41.59461573,  41.15428304,  41.05143588,  40.78110103,
        40.72751156,  40.70432942,  40.3669702 ,  40.1937926 ,
        40.04740289,  39.91532354,  39.73765189,  39.64392867,
        39.32533137,  39.24193123,  39.14934602,  39.11

In [11]:
Sigma.shape

(100,)

In [12]:
VT

array([[ 1.01649501e-03,  3.24983582e-03,  6.00966105e-04, ...,
         3.09477439e-03,  3.08148682e-07,  7.98173946e-07],
       [-1.09329831e-03,  1.97624804e-03, -9.28313176e-04, ...,
        -7.24069029e-03, -8.41308491e-07, -2.17701454e-06],
       [-7.29722782e-04, -3.61007029e-04, -2.97223461e-04, ...,
        -2.08465194e-03,  1.47272235e-07, -7.90891136e-07],
       ...,
       [-1.12641149e-03, -3.40337924e-03, -6.26001220e-04, ...,
        -4.32205013e-03,  1.31299401e-05,  6.35498304e-05],
       [ 3.82979199e-03, -4.38783484e-03, -2.46702872e-03, ...,
        -3.04233310e-03,  1.63561993e-05,  2.72739627e-05],
       [-2.06516483e-03, -2.94601939e-03, -8.17868305e-04, ...,
         1.94412733e-03,  8.89334948e-06,  3.95782407e-05]])

In [13]:
VT.shape

(100, 20635)

### Truncating the number of singular values introduces an approximation which allows to fill the missing urm entries

### Computing a prediction

In [14]:
# Store an intermediate pre-multiplied matrix
import scipy.sparse as sps


s_Vt = sps.diags(Sigma)*VT

In [17]:
prediction = U[user_index, :].dot(s_Vt[:,item_index])

print("Prediction is {:.2f}".format(prediction))

Prediction is -0.00


In [18]:
item_scores = U[user_index, :].dot(s_Vt)
item_scores

array([ 1.02818659e-03,  8.51895450e-04, -1.32128459e-03, ...,
        6.28031499e-03, -7.45080900e-06, -5.65298238e-05])

In [19]:
item_scores.shape

(20635,)

# Let's compare the three MF: BPR, FunkSVD and PureSVD

In [20]:
from MatrixFactorization.Cython.MatrixFactorization_Cython import MatrixFactorization_BPR_Cython, MatrixFactorization_FunkSVD_Cython
from MatrixFactorization.PureSVD import PureSVDRecommender

from Base.Evaluation.Evaluator import SequentialEvaluator

evaluator_test = SequentialEvaluator(URM_test, cutoff_list=[10])

evaluator_validation_early_stopping = SequentialEvaluator(URM_train, cutoff_list=[10], exclude_seen = False)


In [21]:
recommender = MatrixFactorization_BPR_Cython(URM_train)
recommender.fit(num_factors = 50, 
                validation_every_n = 10, 
                stop_on_validation = True, 
                evaluator_object = evaluator_validation_early_stopping,
                lower_validatons_allowed = 5, 
                validation_metric = "MAP")

result_dict, _ = evaluator_test.evaluateRecommender(recommender)

Compiling in Cython
Compiled module saved in subfolder: /MatrixFactorization/Cython
Compilation Complete
Processed 50446 ( 100.00% ) in 0.71 seconds. BPR loss 5.50E-02. Sample per second: 70602
MF_BPR: Epoch 1 of 300. Elapsed time 0.00 min
Processed 50446 ( 100.00% ) in 0.77 seconds. BPR loss 5.50E-02. Sample per second: 65704
MF_BPR: Epoch 2 of 300. Elapsed time 0.00 min
Processed 50446 ( 100.00% ) in 0.83 seconds. BPR loss 5.56E-02. Sample per second: 60729
MF_BPR: Epoch 3 of 300. Elapsed time 0.00 min
Processed 50446 ( 100.00% ) in 0.88 seconds. BPR loss 5.65E-02. Sample per second: 57031
MF_BPR: Epoch 4 of 300. Elapsed time 0.00 min
Processed 50446 ( 100.00% ) in 0.94 seconds. BPR loss 5.96E-02. Sample per second: 53692
MF_BPR: Epoch 5 of 300. Elapsed time 0.01 min
Processed 50446 ( 100.00% ) in 0.99 seconds. BPR loss 6.20E-02. Sample per second: 50936
MF_BPR: Epoch 6 of 300. Elapsed time 0.01 min
Processed 50446 ( 100.00% ) in 1.05 seconds. BPR loss 6.60E-02. Sample per second: 47

KeyboardInterrupt: 

In [ ]:
result_dict

In [ ]:
recommender = MatrixFactorization_FunkSVD_Cython(URM_train)
recommender.fit(num_factors = 50, 
                validation_every_n = 10, 
                stop_on_validation = True, 
                evaluator_object = evaluator_validation_early_stopping,
                lower_validatons_allowed = 5, 
                validation_metric = "MAP")

result_dict, _ = evaluator_test.evaluateRecommender(recommender)

Compiling in Cython
Compiled module saved in subfolder: /MatrixFactorization/Cython
Compilation Complete
Processed 1160777 ( 100.00% ) in 1.35 seconds. MSE loss 8.72E-01. Sample per second: 860400
FUNK_SVD: Epoch 1 of 300. Elapsed time 0.02 min
Processed 1160777 ( 100.00% ) in 1.24 seconds. MSE loss 5.13E-01. Sample per second: 937677
FUNK_SVD: Epoch 2 of 300. Elapsed time 0.03 min
Processed 1160777 ( 100.00% ) in 1.11 seconds. MSE loss 3.51E-01. Sample per second: 1044533
FUNK_SVD: Epoch 3 of 300. Elapsed time 0.04 min
Processed 1160777 ( 100.00% ) in 0.97 seconds. MSE loss 2.73E-01. Sample per second: 1191009
FUNK_SVD: Epoch 4 of 300. Elapsed time 0.06 min
Processed 1160777 ( 100.00% ) in 1.84 seconds. MSE loss 2.39E-01. Sample per second: 630856
FUNK_SVD: Epoch 5 of 300. Elapsed time 0.07 min
Processed 1160777 ( 100.00% ) in 1.77 seconds. MSE loss 2.19E-01. Sample per second: 657257
FUNK_SVD: Epoch 6 of 300. Elapsed time 0.09 min
Processed 1160777 ( 100.00% ) in 1.53 seconds. MSE lo

In [ ]:
recommender = PureSVDRecommender(URM_train)
recommender.fit()

result_dict, _ = evaluator_test.evaluateRecommender(recommender)